In [28]:
import os
import pandas as pd
from dotenv import load_dotenv
from FiinQuantX import FiinSession
from constant import *

load_dotenv()
username = os.getenv('username')
password = os.getenv('password')



client = FiinSession(
    username=username,
    password=password,
).login()

#tickers = VNINDEX # list of tickers: ['UPCOMINDEX',VNINDEX,HNXINDEX]

tickers =client.TickerList(ticker="VNINDEX")

print(tickers)


['CCC', 'SBG', 'FUCTVGF3', 'FUEIP100', 'GMH', 'FUEKIV30', 'NO1', 'FUCTVGF4', 'RYG', 'FUEDCMID', 'FUEKIVFS', 'FUEMAVND', 'FUEFCV50', 'FUEBFVND', 'FUCTVGF5', 'FUEKIVND', 'FUEABVND', 'FUETCC50', 'FUETPVND', 'AAA', 'AAM', 'AAT', 'ABR', 'ABS', 'ABT', 'ACB', 'ACC', 'ACL', 'ADP', 'AGR', 'AGG', 'ACG', 'ATP', 'ANV', 'APG', 'APH', 'HII', 'ASG', 'ASM', 'ASP', 'BAF', 'BBC', 'BCE', 'BCG', 'BFC', 'BHN', 'BIC', 'BID', 'BCM', 'DBD', 'BWE', 'BKG', 'BMC', 'BMI', 'BMP', 'BRC', 'BSI', 'BSR', 'BTP', 'BTT', 'BVH', 'TNH', 'C32', 'C47', 'CCI', 'CCL', 'CDC', 'CRE', 'STK', 'CHP', 'CIG', 'CII', 'CKG', 'CLC', 'ADG', 'CLL', 'CLW', 'CMG', 'CMV', 'CMX', 'CNG', 'COM', 'CRC', 'CSM', 'CSV', 'CTD', 'CTF', 'CTG', 'CTI', 'ICT', 'CTR', 'CTS', 'CVT', 'D2D', 'DAH', 'ADS', 'DPG', 'DBC', 'DBT', 'DC4', 'DCL', 'DCM', 'DGC', 'DGW', 'DHA', 'DHC', 'DHG', 'DHM', 'TTE', 'DIG', 'DLG', 'DMC', 'DSC', 'DSE', 'DPM', 'DPR', 'DQC', 'DRC', 'DRH', 'DRL', 'DSN', 'DTA', 'DTL', 'DTT', 'DVP', 'DXG', 'DXS', 'DXV', 'FUESSV50', 'E1VFVN30', 'EIB', 'E

In [33]:
data = client.Fetch_Trading_Data(
    realtime = False, 
    tickers = tickers,
    fields = ['open', 'high', 'low', 'close', 'volume', 'bu','sd', 'fb', 'fs', 'fn'], 
    from_date='2018-10-01',
    to_date='2022-10-08',
    by = '1d',
).get_data()

df = pd.DataFrame(data)
df.to_csv('vnindex_.csv', index=False)

Fetching data, it may take a while. Please wait...


In [34]:
fi = client.FiinIndicator()
df['sma_20'] = fi.sma(df['close'], window = 20)
df['sma_50'] = fi.sma(df['close'], window = 50)
df['sma_200'] = fi.sma(df['close'], window = 200)
df.to_csv('vnindex_.csv', index=False)

In [58]:
class backtest_strategy:
    def __init__(self, data):
        self.data = data
        self.position = 0  # 1 for holding a position, 0 for no position
        self.cash = 100000  # starting cash
        self.shares = 0  # number of shares held
        self.trades = []  # record of trades

    def run(self):
        for i in range(1, len(self.data)):
            if self.data['sma_20'].iloc[i] > self.data['sma_50'].iloc[i] and self.position == 0:
                # Buy signal
                self.shares = self.cash // self.data['close'].iloc[i]
                self.cash -= self.shares * self.data['close'].iloc[i]
                self.position = 1
                self.trades.append((self.data.index[i], 'BUY', self.shares, self.data['close'].iloc[i]))
                print(f"[BUY] Date: {self.data['timestamp'].iloc[i]}, Shares: {self.shares}, Price: {self.data['close'].iloc[i]}, Cash left: {self.cash}")
            elif self.data['sma_20'].iloc[i] < self.data['sma_50'].iloc[i] and self.position == 1:
                # Sell signal
                self.cash += self.shares * self.data['close'].iloc[i]
                self.position = 0
                self.trades.append((self.data.index[i], 'SELL', self.shares, self.data['close'].iloc[i]))
                print(f"[SELL] Date: {self.data['timestamp'].iloc[i]}, Shares: {self.shares}, Price: {self.data['close'].iloc[i]}, Cash after sale: {self.cash}")
                self.shares = 0

        # Final portfolio value
        final_value = self.cash + (self.shares * self.data['close'].iloc[-1])
        return final_value, self.trades

bt = backtest_strategy(data)
final_value, trades = bt.run()
print(f"Final Portfolio Value: {final_value}")

[BUY] Date: 2021-11-08 00:00, Shares: 6.0, Price: 15664.07217312, Cash left: 6015.566961280012
[SELL] Date: 2021-12-27 00:00, Shares: 6.0, Price: 15546.82612392, Cash after sale: 99296.52370480001
[BUY] Date: 2022-01-04 00:00, Shares: 6.0, Price: 16133.05636992, Cash left: 2498.185485280017
[SELL] Date: 2022-01-06 00:00, Shares: 6.0, Price: 15828.216642, Cash after sale: 97467.48533728001
[BUY] Date: 2022-01-24 00:00, Shares: 6.0, Price: 16109.60716008, Cash left: 809.8423768000066
[SELL] Date: 2022-03-11 00:00, Shares: 6.0, Price: 15335.78323536, Cash after sale: 92824.54178896
[BUY] Date: 2022-07-21 00:00, Shares: 6.0, Price: 14450.5755639, Cash left: 6121.088405560004
[SELL] Date: 2022-07-25 00:00, Shares: 6.0, Price: 14098.8374163, Cash after sale: 90714.11290336002
[BUY] Date: 2022-08-08 00:00, Shares: 6.0, Price: 14509.1985885, Cash left: 3658.921372360026
[SELL] Date: 2022-09-16 00:00, Shares: 6.0, Price: 13571.2301949, Cash after sale: 85086.30254176003
[BUY] Date: 2022-12-02 0

In [31]:
from constant import *
print(len(VNINDEX))

415
